# Traffic Accidents Analysis (India)

**Ishak Islam** | UMID28072552431 | Unified Mentor Internship

---

This notebook analyzes road traffic accidents in India using official government data from the Ministry of Road Transport and Highways (MoRTH). The analysis covers state-wise statistics, collision types, traffic violations, safety device usage, and fatality patterns.

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Define paths
DATA_DIR = Path('../data')
VIZ_DIR = Path('../visualizations')
TABLEAU_DIR = Path('../tableau')

VIZ_DIR.mkdir(exist_ok=True)
TABLEAU_DIR.mkdir(exist_ok=True)

# Color palette
COLORS = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12', '#9b59b6', '#1abc9c', '#e67e22', '#34495e']

print('Setup complete!')

## 2. Load Data

**Dataset Source:** https://data.opencity.in/dataset/road-accidents-in-india-2023

This data is from the Ministry of Road Transport and Highways (MoRTH), Government of India.

In [ ]:
# Load state-wise accidents data
try:
    accidents_df = pd.read_csv(DATA_DIR / 'state_wise_accidents.csv')
    print(f'State accidents data: {accidents_df.shape}')
except FileNotFoundError:
    print('File not found: state_wise_accidents.csv')
    print('Please download from: https://data.opencity.in/dataset/road-accidents-in-india-2023')
    accidents_df = None

In [ ]:
# Load state-wise fatalities data
try:
    fatalities_df = pd.read_csv(DATA_DIR / 'state_wise_fatalities.csv')
    print(f'State fatalities data: {fatalities_df.shape}')
except FileNotFoundError:
    print('File not found: state_wise_fatalities.csv')
    fatalities_df = None

In [ ]:
# Load collision types data
try:
    collision_df = pd.read_csv(DATA_DIR / 'collision_types.csv')
    print(f'Collision types data: {collision_df.shape}')
except FileNotFoundError:
    print('File not found: collision_types.csv')
    collision_df = None

In [ ]:
# Load violations data
try:
    violations_df = pd.read_csv(DATA_DIR / 'violations.csv')
    print(f'Violations data: {violations_df.shape}')
except FileNotFoundError:
    print('File not found: violations.csv')
    violations_df = None

In [ ]:
# Load safety devices data
try:
    safety_df = pd.read_csv(DATA_DIR / 'safety_devices.csv')
    print(f'Safety devices data: {safety_df.shape}')
except FileNotFoundError:
    print('File not found: safety_devices.csv')
    safety_df = None

In [ ]:
# Load road users fatalities data
try:
    road_users_df = pd.read_csv(DATA_DIR / 'road_users_fatalities.csv')
    print(f'Road users data: {road_users_df.shape}')
except FileNotFoundError:
    print('File not found: road_users_fatalities.csv')
    road_users_df = None

## 3. Data Exploration

In [ ]:
# Preview state accidents data
if accidents_df is not None:
    print('State-wise Accidents Data:')
    print(f'Columns: {accidents_df.columns.tolist()}')
    display(accidents_df.head(10))

In [ ]:
# Preview state fatalities data
if fatalities_df is not None:
    print('State-wise Fatalities Data:')
    print(f'Columns: {fatalities_df.columns.tolist()}')
    display(fatalities_df.head(10))

In [ ]:
# Preview collision types data
if collision_df is not None:
    print('Collision Types Data:')
    print(f'Columns: {collision_df.columns.tolist()}')
    display(collision_df)

In [ ]:
# Preview violations data
if violations_df is not None:
    print('Violations Data:')
    print(f'Columns: {violations_df.columns.tolist()}')
    display(violations_df)

In [ ]:
# Preview safety devices data
if safety_df is not None:
    print('Safety Devices Data:')
    print(f'Columns: {safety_df.columns.tolist()}')
    display(safety_df)

In [ ]:
# Preview road users data
if road_users_df is not None:
    print('Road Users Fatalities Data:')
    print(f'Columns: {road_users_df.columns.tolist()}')
    display(road_users_df)

## 4. Data Cleaning

In [ ]:
def clean_state_data(df):
    """Clean state-wise data by removing total rows and handling missing values."""
    if df is None:
        return None
    
    df_clean = df.copy()
    
    # Find the state column
    state_col = None
    for col in df_clean.columns:
        if 'state' in col.lower() or 'ut' in col.lower():
            state_col = col
            break
    
    # Remove total rows
    if state_col:
        df_clean = df_clean[~df_clean[state_col].str.contains('Total|All India|Grand', case=False, na=False)]
    
    # Fill missing numeric values
    numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
    df_clean[numeric_cols] = df_clean[numeric_cols].fillna(0)
    
    return df_clean

# Clean state data
accidents_clean = clean_state_data(accidents_df)
fatalities_clean = clean_state_data(fatalities_df)

if accidents_clean is not None:
    print(f'Cleaned accidents data: {accidents_clean.shape}')
if fatalities_clean is not None:
    print(f'Cleaned fatalities data: {fatalities_clean.shape}')

## 5. Analysis and Statistics

In [ ]:
# Find year columns
def get_year_columns(df):
    """Extract year columns from dataframe."""
    if df is None:
        return []
    year_cols = []
    for col in df.columns:
        try:
            year = int(col)
            if 2000 <= year <= 2030:
                year_cols.append(col)
        except (ValueError, TypeError):
            continue
    return sorted(year_cols)

year_cols = get_year_columns(accidents_clean)
print(f'Year columns found: {year_cols}')

In [ ]:
# Calculate yearly totals
if accidents_clean is not None and year_cols:
    print('Yearly Accident Statistics:')
    print('-' * 40)
    
    for year in year_cols:
        total = accidents_clean[year].sum()
        print(f'{year}: {total:,.0f} accidents')
    
    # Calculate change from first to last year
    if len(year_cols) >= 2:
        first_year = year_cols[0]
        last_year = year_cols[-1]
        change = ((accidents_clean[last_year].sum() - accidents_clean[first_year].sum()) / accidents_clean[first_year].sum()) * 100
        print(f'\nChange from {first_year} to {last_year}: {change:+.1f}%')

In [ ]:
# Calculate yearly fatality totals
if fatalities_clean is not None and year_cols:
    print('Yearly Fatality Statistics:')
    print('-' * 40)
    
    for year in year_cols:
        if year in fatalities_clean.columns:
            total = fatalities_clean[year].sum()
            print(f'{year}: {total:,.0f} fatalities')

In [ ]:
# Find state column
def get_state_column(df):
    """Find the state column name."""
    if df is None:
        return None
    for col in df.columns:
        if 'state' in col.lower() or 'ut' in col.lower():
            return col
    return df.columns[0]

state_col = get_state_column(accidents_clean)
print(f'State column: {state_col}')

In [ ]:
# Top 10 states by accidents (latest year)
if accidents_clean is not None and year_cols:
    latest_year = year_cols[-1]
    print(f'Top 10 States by Accidents ({latest_year}):')
    print('-' * 50)
    
    top_accidents = accidents_clean.nlargest(10, latest_year)[[state_col, latest_year]]
    for i, (_, row) in enumerate(top_accidents.iterrows(), 1):
        print(f'{i}. {row[state_col]}: {row[latest_year]:,.0f}')

In [ ]:
# Top 10 states by fatalities (latest year)
if fatalities_clean is not None and year_cols:
    latest_year = year_cols[-1]
    if latest_year in fatalities_clean.columns:
        print(f'Top 10 States by Fatalities ({latest_year}):')
        print('-' * 50)
        
        fatality_state_col = get_state_column(fatalities_clean)
        top_fatalities = fatalities_clean.nlargest(10, latest_year)[[fatality_state_col, latest_year]]
        for i, (_, row) in enumerate(top_fatalities.iterrows(), 1):
            print(f'{i}. {row[fatality_state_col]}: {row[latest_year]:,.0f}')

## 6. Visualizations

### 6.1 Yearly Trend Analysis

In [ ]:
# Plot yearly trend
if accidents_clean is not None and year_cols:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    totals = [accidents_clean[year].sum() for year in year_cols]
    
    ax.plot(year_cols, totals, marker='o', linewidth=2.5, markersize=10, color=COLORS[0])
    ax.fill_between(year_cols, totals, alpha=0.2, color=COLORS[0])
    
    ax.set_xlabel('Year', fontsize=12)
    ax.set_ylabel('Total Accidents', fontsize=12)
    ax.set_title('Road Accidents Trend in India (2019-2023)', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    # Add value labels
    for year, val in zip(year_cols, totals):
        ax.annotate(f'{val:,.0f}', (year, val), textcoords='offset points', 
                   xytext=(0, 12), ha='center', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(VIZ_DIR / '01_yearly_accidents_trend.png', dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()

In [ ]:
# Plot yearly fatalities trend
if fatalities_clean is not None:
    fatality_year_cols = get_year_columns(fatalities_clean)
    
    if fatality_year_cols:
        fig, ax = plt.subplots(figsize=(12, 6))
        
        totals = [fatalities_clean[year].sum() for year in fatality_year_cols]
        
        ax.plot(fatality_year_cols, totals, marker='s', linewidth=2.5, markersize=10, color=COLORS[1])
        ax.fill_between(fatality_year_cols, totals, alpha=0.2, color=COLORS[1])
        
        ax.set_xlabel('Year', fontsize=12)
        ax.set_ylabel('Total Fatalities', fontsize=12)
        ax.set_title('Road Accident Fatalities Trend in India (2019-2023)', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        
        for year, val in zip(fatality_year_cols, totals):
            ax.annotate(f'{val:,.0f}', (year, val), textcoords='offset points', 
                       xytext=(0, 12), ha='center', fontsize=10, fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(VIZ_DIR / '02_yearly_fatalities_trend.png', dpi=300, bbox_inches='tight', facecolor='white')
        plt.show()

### 6.2 State-wise Analysis

In [ ]:
# Top 10 states by accidents
if accidents_clean is not None and year_cols:
    latest_year = year_cols[-1]
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    top10 = accidents_clean.nlargest(10, latest_year)[[state_col, latest_year]]
    
    bars = ax.barh(range(len(top10)), top10[latest_year].values, color=COLORS[0])
    ax.set_yticks(range(len(top10)))
    ax.set_yticklabels(top10[state_col].values, fontsize=11)
    ax.invert_yaxis()
    
    ax.set_xlabel('Number of Accidents', fontsize=12)
    ax.set_title(f'Top 10 States by Road Accidents ({latest_year})', fontsize=14, fontweight='bold')
    
    # Add value labels
    for bar, val in zip(bars, top10[latest_year].values):
        ax.text(val + 1000, bar.get_y() + bar.get_height()/2, f'{val:,.0f}', 
               va='center', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(VIZ_DIR / '03_top_states_accidents.png', dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()

In [ ]:
# Top 10 states by fatalities
if fatalities_clean is not None:
    fatality_year_cols = get_year_columns(fatalities_clean)
    
    if fatality_year_cols:
        latest_year = fatality_year_cols[-1]
        fatality_state_col = get_state_column(fatalities_clean)
        
        fig, ax = plt.subplots(figsize=(12, 8))
        
        top10 = fatalities_clean.nlargest(10, latest_year)[[fatality_state_col, latest_year]]
        
        bars = ax.barh(range(len(top10)), top10[latest_year].values, color=COLORS[1])
        ax.set_yticks(range(len(top10)))
        ax.set_yticklabels(top10[fatality_state_col].values, fontsize=11)
        ax.invert_yaxis()
        
        ax.set_xlabel('Number of Fatalities', fontsize=12)
        ax.set_title(f'Top 10 States by Road Accident Fatalities ({latest_year})', fontsize=14, fontweight='bold')
        
        for bar, val in zip(bars, top10[latest_year].values):
            ax.text(val + 500, bar.get_y() + bar.get_height()/2, f'{val:,.0f}', 
                   va='center', fontsize=10)
        
        plt.tight_layout()
        plt.savefig(VIZ_DIR / '04_top_states_fatalities.png', dpi=300, bbox_inches='tight', facecolor='white')
        plt.show()

### 6.3 Collision Type Analysis

In [ ]:
# Collision types analysis
if collision_df is not None:
    print('Collision Types Data:')
    display(collision_df)
    
    # Find the type and value columns
    cols = collision_df.columns.tolist()
    type_col = cols[0]
    
    # Look for numeric columns for accidents
    numeric_cols = collision_df.select_dtypes(include=[np.number]).columns.tolist()
    
    if numeric_cols:
        value_col = numeric_cols[0]
        
        fig, ax = plt.subplots(figsize=(14, 8))
        
        data = collision_df[[type_col, value_col]].dropna()
        data = data.sort_values(value_col, ascending=True)
        
        colors = [COLORS[i % len(COLORS)] for i in range(len(data))]
        bars = ax.barh(range(len(data)), data[value_col].values, color=colors)
        ax.set_yticks(range(len(data)))
        ax.set_yticklabels(data[type_col].values, fontsize=10)
        
        ax.set_xlabel('Count', fontsize=12)
        ax.set_title('Road Accidents by Collision Type (2023)', fontsize=14, fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(VIZ_DIR / '05_collision_types.png', dpi=300, bbox_inches='tight', facecolor='white')
        plt.show()

### 6.4 Traffic Violations Analysis

In [ ]:
# Violations analysis
if violations_df is not None:
    print('Violations Data:')
    display(violations_df)
    
    cols = violations_df.columns.tolist()
    type_col = cols[0]
    
    numeric_cols = violations_df.select_dtypes(include=[np.number]).columns.tolist()
    
    if numeric_cols:
        value_col = numeric_cols[0]
        
        fig, ax = plt.subplots(figsize=(14, 10))
        
        data = violations_df[[type_col, value_col]].dropna()
        data = data[~data[type_col].str.contains('Total|Grand', case=False, na=False)]
        data = data.sort_values(value_col, ascending=True).tail(15)
        
        colors = [COLORS[i % len(COLORS)] for i in range(len(data))]
        bars = ax.barh(range(len(data)), data[value_col].values, color=colors)
        ax.set_yticks(range(len(data)))
        ax.set_yticklabels(data[type_col].values, fontsize=10)
        
        ax.set_xlabel('Number of Accidents', fontsize=12)
        ax.set_title('Road Accidents by Traffic Violation Type (2023)', fontsize=14, fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(VIZ_DIR / '06_violations.png', dpi=300, bbox_inches='tight', facecolor='white')
        plt.show()

### 6.5 Safety Device Analysis

In [ ]:
# Safety devices analysis
if safety_df is not None:
    print('Safety Devices Data:')
    display(safety_df)
    
    cols = safety_df.columns.tolist()
    
    if len(cols) >= 2:
        type_col = cols[0]
        numeric_cols = safety_df.select_dtypes(include=[np.number]).columns.tolist()
        
        if numeric_cols:
            value_col = numeric_cols[0]
            
            fig, axes = plt.subplots(1, 2, figsize=(16, 7))
            
            data = safety_df[[type_col, value_col]].dropna()
            data = data[~data[type_col].str.contains('Total|Grand', case=False, na=False)]
            
            # Pie chart
            if len(data) > 0:
                axes[0].pie(data[value_col].values, labels=data[type_col].values, 
                           autopct='%1.1f%%', colors=COLORS[:len(data)], explode=[0.02]*len(data))
                axes[0].set_title('Distribution of Accidents by Safety Device Absence', fontsize=12, fontweight='bold')
            
            # Bar chart
            bars = axes[1].barh(range(len(data)), data[value_col].values, color=COLORS[:len(data)])
            axes[1].set_yticks(range(len(data)))
            axes[1].set_yticklabels(data[type_col].values, fontsize=10)
            axes[1].set_xlabel('Number of Accidents', fontsize=12)
            axes[1].set_title('Accidents by Missing Safety Device Type', fontsize=12, fontweight='bold')
            axes[1].invert_yaxis()
            
            plt.tight_layout()
            plt.savefig(VIZ_DIR / '07_safety_devices.png', dpi=300, bbox_inches='tight', facecolor='white')
            plt.show()

### 6.6 Road User Fatalities Analysis

In [ ]:
# Road users fatalities analysis
if road_users_df is not None:
    print('Road Users Fatalities Data:')
    display(road_users_df)
    
    cols = road_users_df.columns.tolist()
    
    if len(cols) >= 2:
        type_col = cols[0]
        numeric_cols = road_users_df.select_dtypes(include=[np.number]).columns.tolist()
        
        if numeric_cols:
            value_col = numeric_cols[0]
            
            fig, ax = plt.subplots(figsize=(12, 10))
            
            data = road_users_df[[type_col, value_col]].dropna()
            data = data[~data[type_col].str.contains('Total|Grand', case=False, na=False)]
            data = data[data[value_col] > 0]
            
            if len(data) > 0:
                colors = COLORS[:len(data)] if len(data) <= len(COLORS) else [COLORS[i % len(COLORS)] for i in range(len(data))]
                wedges, texts, autotexts = ax.pie(data[value_col].values, labels=data[type_col].values, 
                                                  autopct='%1.1f%%', colors=colors, 
                                                  explode=[0.03]*len(data), pctdistance=0.75)
                
                for autotext in autotexts:
                    autotext.set_fontsize(9)
                for text in texts:
                    text.set_fontsize(9)
            
            ax.set_title('Road Accident Fatalities by Road User Type (2023)', fontsize=14, fontweight='bold')
            
            plt.tight_layout()
            plt.savefig(VIZ_DIR / '08_road_users_fatalities.png', dpi=300, bbox_inches='tight', facecolor='white')
            plt.show()

### 6.7 State Comparison

In [ ]:
# State comparison: Accidents vs Fatalities
if accidents_clean is not None and fatalities_clean is not None and year_cols:
    latest_year = year_cols[-1]
    
    if latest_year in accidents_clean.columns and latest_year in fatalities_clean.columns:
        fig, axes = plt.subplots(1, 2, figsize=(18, 8))
        
        # Top 10 by accidents
        top_accidents = accidents_clean.nlargest(10, latest_year)[[state_col, latest_year]]
        bars1 = axes[0].barh(range(len(top_accidents)), top_accidents[latest_year].values, color=COLORS[0])
        axes[0].set_yticks(range(len(top_accidents)))
        axes[0].set_yticklabels(top_accidents[state_col].values, fontsize=11)
        axes[0].invert_yaxis()
        axes[0].set_xlabel('Number of Accidents', fontsize=12)
        axes[0].set_title(f'Top 10 States by Accidents ({latest_year})', fontsize=13, fontweight='bold')
        
        for bar, val in zip(bars1, top_accidents[latest_year].values):
            axes[0].text(val + 500, bar.get_y() + bar.get_height()/2, f'{val:,.0f}', va='center', fontsize=9)
        
        # Top 10 by fatalities
        fatality_state_col = get_state_column(fatalities_clean)
        top_fatalities = fatalities_clean.nlargest(10, latest_year)[[fatality_state_col, latest_year]]
        bars2 = axes[1].barh(range(len(top_fatalities)), top_fatalities[latest_year].values, color=COLORS[1])
        axes[1].set_yticks(range(len(top_fatalities)))
        axes[1].set_yticklabels(top_fatalities[fatality_state_col].values, fontsize=11)
        axes[1].invert_yaxis()
        axes[1].set_xlabel('Number of Fatalities', fontsize=12)
        axes[1].set_title(f'Top 10 States by Fatalities ({latest_year})', fontsize=13, fontweight='bold')
        
        for bar, val in zip(bars2, top_fatalities[latest_year].values):
            axes[1].text(val + 300, bar.get_y() + bar.get_height()/2, f'{val:,.0f}', va='center', fontsize=9)
        
        plt.suptitle('State-wise Comparison: Accidents vs Fatalities', fontsize=15, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.savefig(VIZ_DIR / '09_state_comparison.png', dpi=300, bbox_inches='tight', facecolor='white')
        plt.show()

## 7. Export Data for Tableau

In [ ]:
# Export state accidents data
if accidents_clean is not None:
    accidents_clean.to_csv(TABLEAU_DIR / 'state_accidents_tableau.csv', index=False)
    print(f'Exported state accidents: {len(accidents_clean)} rows')

# Export state fatalities data
if fatalities_clean is not None:
    fatalities_clean.to_csv(TABLEAU_DIR / 'state_fatalities_tableau.csv', index=False)
    print(f'Exported state fatalities: {len(fatalities_clean)} rows')

In [ ]:
# Export collision types data
if collision_df is not None:
    collision_df.to_csv(TABLEAU_DIR / 'collision_types_tableau.csv', index=False)
    print(f'Exported collision types: {len(collision_df)} rows')

# Export violations data
if violations_df is not None:
    violations_df.to_csv(TABLEAU_DIR / 'violations_tableau.csv', index=False)
    print(f'Exported violations: {len(violations_df)} rows')

In [ ]:
# Export safety devices data
if safety_df is not None:
    safety_df.to_csv(TABLEAU_DIR / 'safety_devices_tableau.csv', index=False)
    print(f'Exported safety devices: {len(safety_df)} rows')

# Export road users data
if road_users_df is not None:
    road_users_df.to_csv(TABLEAU_DIR / 'road_users_tableau.csv', index=False)
    print(f'Exported road users: {len(road_users_df)} rows')

In [ ]:
# Create summary data for Tableau
if accidents_clean is not None and year_cols:
    summary_data = {
        'Year': year_cols,
        'Total_Accidents': [accidents_clean[year].sum() for year in year_cols]
    }
    
    if fatalities_clean is not None:
        summary_data['Total_Fatalities'] = [fatalities_clean[year].sum() if year in fatalities_clean.columns else 0 for year in year_cols]
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv(TABLEAU_DIR / 'yearly_summary_tableau.csv', index=False)
    print(f'Exported yearly summary')
    display(summary_df)

## 8. Key Findings

In [ ]:
print('=' * 60)
print('KEY FINDINGS - TRAFFIC ACCIDENTS ANALYSIS (INDIA)')
print('=' * 60)

if accidents_clean is not None and year_cols:
    latest_year = year_cols[-1]
    first_year = year_cols[0]
    
    total_latest = accidents_clean[latest_year].sum()
    total_first = accidents_clean[first_year].sum()
    change = ((total_latest - total_first) / total_first) * 100
    
    print(f'\n1. OVERALL TREND ({first_year}-{latest_year})')
    print(f'   - Total accidents in {latest_year}: {total_latest:,.0f}')
    print(f'   - Change from {first_year}: {change:+.1f}%')
    
    print(f'\n2. TOP STATE ({latest_year})')
    top_state = accidents_clean.nlargest(1, latest_year)
    print(f'   - {top_state[state_col].values[0]}: {top_state[latest_year].values[0]:,.0f} accidents')

if fatalities_clean is not None:
    fatality_year_cols = get_year_columns(fatalities_clean)
    if fatality_year_cols:
        latest = fatality_year_cols[-1]
        total_deaths = fatalities_clean[latest].sum()
        print(f'\n3. TOTAL FATALITIES ({latest})')
        print(f'   - {total_deaths:,.0f} people died in road accidents')

print('\n4. DATA SOURCES')
print('   - Ministry of Road Transport and Highways (MoRTH)')
print('   - OpenCity Data Portal')

print('\n' + '=' * 60)

## 9. Recommendations

In [ ]:
print('RECOMMENDATIONS')
print('-' * 40)
print('''
Based on the analysis, the following recommendations are suggested:

1. FOCUS ON HIGH-RISK STATES
   - Target road safety campaigns in states with highest accident rates
   - Improve road infrastructure in accident-prone areas

2. ENFORCE SAFETY DEVICE USAGE
   - Strict helmet and seatbelt enforcement
   - Increase awareness about safety device importance

3. ADDRESS MAJOR VIOLATIONS
   - Focus on speeding, drunk driving, and wrong-side driving
   - Implement better traffic monitoring systems

4. PROTECT VULNERABLE ROAD USERS
   - Improve pedestrian infrastructure
   - Create dedicated lanes for two-wheelers and cyclists

5. DATA-DRIVEN POLICY MAKING
   - Use state-wise data for targeted interventions
   - Monitor trends for measuring effectiveness of safety measures
''')

## 10. Summary

In [ ]:
print('=' * 60)
print('ANALYSIS SUMMARY')
print('=' * 60)
print('''
This analysis examined road traffic accidents in India using
official government data from the Ministry of Road Transport
and Highways.

Key aspects covered:
- State-wise accident and fatality trends (2019-2023)
- Collision type analysis
- Traffic violation patterns
- Safety device compliance
- Road user fatality distribution

Outputs generated:
- 9 visualizations (PNG files)
- 7 data exports for Tableau dashboards

Data Source: https://data.opencity.in/dataset/road-accidents-in-india-2023
''')
print('=' * 60)
print('\nAnalysis completed by: Ishak Islam')
print('Internship ID: UMID28072552431')
print('Program: Unified Mentor Internship')
print('=' * 60)